In [1]:
import fsps

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from ChangTools.plotting import prettyplot
from ChangTools.plotting import prettycolors

In [3]:
prettyplot()
pretty_colors = prettycolors()

In [ ]:
sps = fsps.StellarPopulation(compute_vega_mags=False, zcontinuous=1, sfh=0, logzsol=0.0, dust_type=2, dust2=0.2)

In [ ]:
fig = plt.figure()
sub = fig.add_subplot(111)
sps.params['sfh'] = 1
for tau in [1., 5., 10., 50]:
    sps.params['tau'] = tau
    sps.params['const'] = 0.5
    sps.params['sf_start'] = 5.
    wave, spec = sps.get_spectrum(tage=1.)
    print spec
    sub.plot(wave, 1.e17*spec)

sub.set_xlim([1000, 10000])
